In [1]:
import pandas as pd
import numpy as np
import requests
import json

Создадим функцию, которая будет определять сигнатуру по анализу DE.

In [6]:
def make_signature_from_DE (file, logFC = (-1, 1), pvalue = 0.01):
    """
    find genes with increased and decreased expression
    
    Parametrs
    ---------
    file: the path to the file with differential expression results (DESeq2, EdgeR)
    logFC: a tuple with a lower and upper threshold for log2 fold change
    pvalue: threshold for pvalue
    
    Return
    ------
    a tuple with 2 lists: 1) list of genes with increased expression - genes with a logarithm of the fold change greater than 
                            set upper threshold for logFC, pvalue less than threshold for pvalue
                          2) list of genes with reduced  expression - genes with a logarithm of the fold change less than 
                            set lower threshold for logFC, pvalue less than threshold for pvalue
    Each list of genes is sorted decrease of the modulo the logarithm of the fold change and sorted by increasing pvalue.
    """
    Dif_exp = pd.read_table(file, sep = '\t')
    
    if ('logFC' in Dif_exp.columns.tolist()) & ('PValue' in Dif_exp.columns.tolist()):# названия столбцов 'logFC', PValue' характерны для edgeR
        Dif_exp_up = Dif_exp[(Dif_exp['logFC'] > logFC[1]) & (Dif_exp['PValue'] < pvalue)]
        Dif_exp_up = Dif_exp_up.sort_values(by = ['logFC', 'PValue'], ascending= [False, True])
        Dif_exp_down = Dif_exp[(Dif_exp['logFC'] < logFC[0]) & (Dif_exp['PValue'] < pvalue)]
        Dif_exp_down = Dif_exp_down.sort_values(by = ['logFC', 'PValue'], ascending = [True, True])
        Dif_exp_up_genes = Dif_exp_up['logFC']
        Dif_exp_down_genes = Dif_exp_down['logFC']
    elif ('log2FoldChange' in Dif_exp.columns.tolist()) & ('pvalue' in Dif_exp.columns.tolist()):# названия столбцов 'log2FoldChange', 'pvalue' характерны для DESeq2
        Dif_exp_up = Dif_exp[(Dif_exp['log2FoldChange'] > logFC[1]) & (Dif_exp['pvalue'] < pvalue)]
        Dif_exp_up = Dif_exp_up.sort_values(by = ['log2FoldChange', 'pvalue'], ascending = [False, True])
        Dif_exp_down = Dif_exp[(Dif_exp['log2FoldChange'] < logFC[0]) & (Dif_exp['pvalue'] < pvalue)]
        Dif_exp_down = Dif_exp_down.sort_values(by = ['log2FoldChange', 'pvalue'], ascending = [True, True])
        Dif_exp_up_genes = Dif_exp_up['log2FoldChange']
        Dif_exp_down_genes = Dif_exp_down['log2FoldChange']
        

    return (Dif_exp_up_genes, Dif_exp_down_genes)

Для примера посмотрим на результаты анализа дифференциальной экспрессии для образцов клеток сердца и фибробластов c помощью edgeR.

Посмотрим сам файл с результатами.

In [3]:
data = pd.read_table('DATA/DE_heart_fibroblast/DE_with_edgeR_for_heart_fibroblast.txt', sep = '\t')
data.head()

,logFC,logCPM,PValue
A1BG,-1.616721,3.216281,3.648526e-26
A1CF,0.733942,-1.202507,2.740174e-04
A2M,3.911215,8.547616,7.819607e-114
A2ML1,-1.043900,0.707597,2.489561e-12
A2MP1,6.370985,1.889518,2.858788e-226


Определим гены с повышенной и пониженной экспрессией.

In [42]:
data_up_genes, data_down_genes = make_signature_from_DE('DATA/DE_heart_fibroblast/DE_with_edgeR_for_heart_fibroblast.txt',  logFC = (-1.5, 1.5), pvalue = 0.01)
data_up_genes.shape

(5290,)

In [18]:
data_up_genes.head()

MYH7      18.790898
MYL2      17.093563
MYH6      16.669100
MB        15.254694
MYBPC3    15.228322
Name: logFC, dtype: float64

In [44]:
data_up_genes = pd.DataFrame(data_up_genes)
data_up_genes.head()

,logFC
MYH7,18.790898
MYL2,17.093563
MYH6,16.669100
MB,15.254694
MYBPC3,15.228322


In [21]:
data_up = pd.read_csv('DATA/protein_network/df_topolog_metrics_up.csv', index_col = 0)
data_up.head()

,betweenness,pagerank,closeness,katz,hits_authority,hits_hub,eigenvector
MYH7,0.000318,0.002122,0.166280,0.023405,1.383970e-06,1.383970e-06,1.406593e-06
MYL2,0.000000,0.001015,0.142857,0.022700,1.918408e-07,1.918408e-07,1.968299e-07
MYH6,0.000000,0.001253,0.165895,0.022954,1.342001e-06,1.342001e-06,1.363082e-06
MB,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00
MYBPC3,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00


In [43]:
data_up.shape

(2000, 7)

In [45]:
data = data_up_genes.merge(data_up, how='left', left_index = True, right_index = True)

In [58]:
data.head()

,logFC,betweenness,pagerank,closeness,katz,hits_authority,hits_hub,eigenvector
MYH7,18.790898,0.000318,0.002122,0.166280,0.023405,1.383970e-06,1.383970e-06,1.406593e-06
MYL2,17.093563,0.000000,0.001015,0.142857,0.022700,1.918408e-07,1.918408e-07,1.968299e-07
MYH6,16.669100,0.000000,0.001253,0.165895,0.022954,1.342001e-06,1.342001e-06,1.363082e-06
MB,15.254694,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00
MYBPC3,15.228322,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00


In [61]:
data["inf_score"] = np.ones(data.shape[0])

In [62]:
data.head()

,logFC,betweenness,pagerank,closeness,katz,hits_authority,hits_hub,eigenvector,inf_score
MYH7,18.790898,0.000318,0.002122,0.166280,0.023405,1.383970e-06,1.383970e-06,1.406593e-06,1.0
MYL2,17.093563,0.000000,0.001015,0.142857,0.022700,1.918408e-07,1.918408e-07,1.968299e-07,1.0
MYH6,16.669100,0.000000,0.001253,0.165895,0.022954,1.342001e-06,1.342001e-06,1.363082e-06,1.0
MB,15.254694,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00,1.0
MYBPC3,15.228322,0.000000,0.000212,NaN,0.022237,0.000000e+00,0.000000e+00,0.000000e+00,1.0


In [60]:
list(data.columns)

['logFC',
 'betweenness',
 'pagerank',
 'closeness',
 'katz',
 'hits_authority',
 'hits_hub',
 'eigenvector']

In [57]:
data[not(data.iloc[5200:5230, 2:].isna())]

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [37]:
data.loc['AC000374.1']

logFC             5.393339
betweenness            NaN
pagerank               NaN
closeness              NaN
katz                   NaN
hits_authority         NaN
hits_hub               NaN
eigenvector            NaN
Name: AC000374.1, dtype: float64

In [ ]:
data_up_genes[]

In [9]:
data_down_genes.shape

(6770,)

In [11]:
data_down_genes.head()

MMP3       -12.660765
KRTAP2-3   -11.717598
KRTAP1-5   -11.404216
KRT34      -11.160330
TERT       -10.020205
Name: logFC, dtype: float64

In [15]:
data_up_genes = list(data_up_genes.index)

In [16]:
data_down_genes = list(data_down_genes.index)

Сделаем API запрос.

In [6]:
url = 'https://maayanlab.cloud/L1000CDS2/query'
def upperGenes(genes):
    # The app uses uppercase gene symbols. So it is crucial to perform upperGenes() step.
    return [gene.upper() for gene in genes]

# gene-set search example
data = {"upGenes": data_up_genes,
        "dnGenes": data_down_genes}
data['upGenes'] = upperGenes(data['upGenes'])
data['dnGenes'] = upperGenes(data['dnGenes'])
config = {"aggravate": True, "searchMethod": "geneSet",
          "share": True, "combination": True,
          "db-version": "latest"}

#metadata = [{"key": "Tag", "value": "gene-set python example"}, {"key": "Cell", "value": "MCF7"}]
payload = {"data": data, "config": config}
headers = {'content-type': 'application/json'}
r = requests.post(url, data=json.dumps(payload), headers = headers)
resGeneSet = r.json()

Посмотрим perturbation name, PubChem ID, combinations.

In [7]:
try:
    for i in resGeneSet['topMeta']:
        if 'pert_desc' in i:
            print(i['pert_desc'])
except KeyError:
    print("В запросе нет данного ключа")

PP-110
PP-110
PLX-4032
AS605240
-666
BMS-536924
AS605240
BMS-536924
canertinib
-666
selumetinib
PD-184352
PLX-4720
canertinib
TG101348
TG101348
trametinib
PLX-4032
AZD-8330
PD-0325901
-666
palbociclib
-666
PD-0325901
selumetinib
PHA-665752
selumetinib
BMS-754807
trametinib
BMS-536924
-666
PHA-793887
TGX-221
trametinib
trametinib
PD-0325901
palbociclib
PD-184352
foretinib
PD-0325901
PD-0325901
vorinostat
erlotinib
gefitinib
QUINACRINE HYDROCHLORIDE
AZD-8330
PD-0325901
palbociclib
selumetinib
Nutlin-3


In [8]:
try:
    for i in resGeneSet['topMeta']:
        if 'pert_id' in i:
            print(i['pert_id'])
except KeyError:
    print("В запросе нет данного ключа")

BRD-K03618428
BRD-K03618428
BRD-K56343971
BRD-K41895714
BRD-K57080016
BRD-K34581968
BRD-K41895714
BRD-K34581968
BRD-K50168500
BRD-K53414658
BRD-K57080016
BRD-K05104363
BRD-K16478699
BRD-K50168500
BRD-K12502280
BRD-K12502280
BRD-K12343256
BRD-K56343971
BRD-K37687095
BRD-K49865102
BRD-K68548958
BRD-K51313569
BRD-K57080016
BRD-K49865102
BRD-K57080016
BRD-K95435023
BRD-K57080016
BRD-K13049116
BRD-K12343256
BRD-K34581968
BRD-K98490050
BRD-K64800655
BRD-A41692738
BRD-K12343256
BRD-K12343256
BRD-K49865102
BRD-K51313569
BRD-K05104363
BRD-K03449891
BRD-K49865102
BRD-K49865102
BRD-K81418486
BRD-K70401845
BRD-K64052750
BRD-A45889380
BRD-K37687095
BRD-K49865102
BRD-K51313569
BRD-K57080016
BRD-A12230535


In [9]:
try:
    for i in resGeneSet['topMeta']:
        if 'pubchem_id' in i:
            print(i['pubchem_id'])
except KeyError:
    print("В запросе нет данного ключа")

24905203
24905203
42611257
10377751
10127622
11353973
10377751
11353973
156414
9911830
10127622
6918454
24180719
156414
16722836
16722836
11707110
42611257
16666708
9826528
1285940
5330286
10127622
9826528
10127622
10461815
10127622
24785538
11707110
11353973
3926765
46191454
9907093
11707110
11707110
9826528
5330286
6918454
42642645
9826528
9826528
5311
176871
123631
23581813
16666708
9826528
5330286
10127622
216345


In [10]:
try:
    for i in resGeneSet['combinations']:
        print(i)
except KeyError:
    print("В запросе нет данного ключа")

{'X1': 'CPC006_HT29_24H:BRD-K03618428:22.2', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0648}
{'X1': 'CPC006_A375_24H:BRD-K03618428:22.2', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0615}
{'X1': 'CPC006_A375_24H:BRD-K56343971:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0609}
{'X1': 'CPC006_A375_24H:BRD-K57080016:80.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0606}
{'X1': 'CPC006_A375_24H:BRD-K41895714:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0604}
{'X1': 'CPC006_A375_24H:BRD-K34581968:11.1', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.06}
{'X1': 'CPC006_HT29_24H:BRD-K41895714:10.0', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0598}
{'X1': 'CPC006_HT29_24H:BRD-K34581968:11.1', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0585}
{'X1': 'LJP005_HT29_24H:BRD-K05104363:10', 'X2': 'CPC006_HA1E_24H:BRD-K81418486:10.0', 'value': 0.0574}
{'X1': 'CPC006_A375_24H:BRD-K53414658:10.0', 'X2':